In [1]:
import numpy as np
import pandas as pd

In [2]:
sample_train_data = pd.read_csv("/workspaces/NLP---Text-Classification-of-Coronavirus-Tweets/dataset/Corona_NLP_train.csv", encoding = 'latin1')
sample_test_data = pd.read_csv("/workspaces/NLP---Text-Classification-of-Coronavirus-Tweets/dataset/Corona_NLP_test.csv", encoding = 'latin1')

In [3]:
sample_train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [4]:
sample_test_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [5]:
sample_train_data["Sentiment"].value_counts()

Sentiment
Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: count, dtype: int64

In [6]:
sample_test_data["Sentiment"].value_counts()

Sentiment
Negative              1041
Positive               947
Neutral                619
Extremely Positive     599
Extremely Negative     592
Name: count, dtype: int64

so it is interesting to note that dataset is more or less balanced for training and testing so there is not a huge class imbalance. Thus we can first start with the usual modelling methodology and then move to the class weighting if necessary

In [7]:
training_tweets = sample_train_data["OriginalTweet"]
training_labels = sample_train_data["Sentiment"]

testing_tweets = sample_test_data["OriginalTweet"]
testing_labels = sample_test_data["Sentiment"]

In [8]:
max_length = 128
padding = "max_length"
truncation = True

import torch
from transformers import AutoTokenizer
from transformers import BertModel


tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa")

/home/codespace/.local/share/virtualenvs/NLP---Text-Classification-of-Coronavirus-T-RKSHQG45/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [12]:
print(f"How the tokenizer vocabulary looks like: {tokenizer.vocab}")
print(f"How many unique tokens are in the tokenizer vocabulary : {tokenizer.vocab_size}")

How the tokenizer vocabulary looks like: {'bled': 23919, 'fender': 19028, 'liberia': 18039, '##ط': 29828, 'cowan': 25752, 'guido': 20239, 'gunshot': 22077, 'peg': 25039, '和': 1796, '##town': 4665, 'mort': 22294, '##lham': 25968, 'deposited': 14140, 'engages': 24255, 'mughal': 17877, '章': 1932, 'sizes': 10826, 'epsilon': 28038, '##hole': 11484, '##hab': 25459, 'dentist': 24385, 'dal': 17488, '[unused40]': 41, 'regulates': 26773, 'hurts': 13403, 'tuesday': 9857, '1914': 4554, 'acquiring': 13868, 'cyclists': 21912, 'corsica': 27047, '##nb': 27698, 'checked': 7039, '167': 16785, 'brethren': 17937, 'reissued': 14805, 'bank': 2924, '##inga': 28234, 'hosting': 9936, 'warriors': 6424, 'mini': 7163, '1796': 13885, 'opened': 2441, '[unused562]': 567, '1781': 16788, 'convenience': 15106, 'roster': 9238, '##yard': 14132, 'loretta': 28493, 'peculiar': 14099, 'connor': 6720, 'frequented': 24832, 'saturdays': 18860, 'integration': 8346, 'spiders': 14160, 'matilda': 17981, '[unused848]': 853, 'william

In [13]:
len(tokenizer.encode(training_tweets[4], padding=padding, truncation=truncation, max_length=max_length))

128

In [27]:
sample_tokens = tokenizer(training_tweets[4], padding=padding, truncation=truncation, max_length=max_length, return_tensors="pt")
sample_embeddings = model.embeddings(sample_tokens['input_ids'])